In [3]:
import pandas as pd

# Load the dataset
data = pd.read_csv('combined_data_zero_sentiment.csv')

# Display the first few rows
data.head()
# Drop rows with missing Ethereum data
data.dropna(subset=['Open_ETH', 'High_ETH', 'Low_ETH', 'Close_ETH', 'Adj Close_ETH', 'Volume_ETH'], inplace=True)

# Fill 'Tweet' column with a placeholder
data['Tweet'].fillna('No Tweet', inplace=True)

# Calculate daily price change for Bitcoin and Ethereum
data['BTC_Daily_Change'] = data['Close'] - data['Open']
data['ETH_Daily_Change'] = data['Close_ETH'] - data['Open_ETH']

# Compute a moving average for the prices of Bitcoin and Ethereum (using a window of 7 days for simplicity)
data['BTC_Moving_Avg_7'] = data['Close'].rolling(window=7).mean()
data['ETH_Moving_Avg_7'] = data['Close_ETH'].rolling(window=7).mean()

# Drop the initial rows where moving average couldn't be computed (first 6 rows)
data.dropna(subset=['BTC_Moving_Avg_7', 'ETH_Moving_Avg_7'], inplace=True)

data.head()
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert 'Tweet' column to numerical data using TF-IDF
vectorizer = TfidfVectorizer(max_features=50)  # Limiting to top 50 features for simplicity
tfidf_matrix = vectorizer.fit_transform(data['Tweet'])

# Convert the matrix to a dataframe to concatenate with our original data
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=data.index)

# Drop the original 'Tweet' column and concatenate the TF-IDF dataframe
data = pd.concat([data.drop(columns=['Tweet']), tfidf_df], axis=1)

# Split the data into features (X) and target (y)
X = data.drop(columns=['Date', 'Close_ETH', 'Adj Close_ETH'])
y = data['Close_ETH']

X.head()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5, :]  # Display the first 5 rows of the scaled training features for inspection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and train the linear regression model
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Predict on both training and test data
y_train_pred = lr.predict(X_train_scaled)
y_test_pred = lr.predict(X_test_scaled)

# Evaluate the model's performance
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_mse, test_mse, train_r2, test_r2


(2.5579208161457934e-24, 370.0280002880646, 1.0, 0.9997673708797274)

In [4]:
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the models
rf = RandomForestRegressor(n_estimators=100, random_state=42)
ada = AdaBoostRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=100, random_state=42)
lr = LinearRegression()

# Create a dictionary to store the models and their names
models = {'Linear Regression': lr, 'Random Forest': rf, 'AdaBoost': ada, 'XGBoost': xgb}

# Loop through the models and fit them to the training data, then evaluate their performance
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions for training and test data
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Metrics for training and test data
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Create scatter plot for each model for both training and test data
    plt.figure(figsize=(14, 7))
    
    plt.subplot(1, 2, 1)
    plt.scatter(y_train, y_train_pred, label='Training Data', alpha=0.5)
    plt.title(f'{name} - Training Data Predictions')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.scatter(y_test, y_test_pred, label='Test Data', alpha=0.5)
    plt.title(f'{name} - Test Data Predictions')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Model: {name}")
    print(f"Train MSE: {train_mse}, Test MSE: {test_mse}")
    print(f"Train R2 Score: {train_r2}, Test R2 Score: {test_r2}")
    print("-" * 30)


NameError: name 'RandomForestRegressor' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the models
rf = RandomForestRegressor(n_estimators=100, random_state=42)
ada = AdaBoostRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=100, random_state=42)
lr = LinearRegression()

# Create a dictionary to store the models and their names
models = {'Linear Regression': lr, 'Random Forest': rf, 'AdaBoost': ada, 'XGBoost': xgb}

# Loop through the models and fit them to the training data, then evaluate their performance
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions for training and test data
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Metrics for training and test data
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Create scatter plot for each model for both training and test data
    plt.figure(figsize=(14, 7))
    
    plt.subplot(1, 2, 1)
    plt.scatter(y_train, y_train_pred, label='Training Data', alpha=0.5)
    plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], 'r--', label='Perfect Prediction')
    plt.title(f'{name} - Training Data Predictions')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.scatter(y_test, y_test_pred, label='Test Data', alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label='Perfect Prediction')
    plt.title(f'{name} - Test Data Predictions')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Model: {name}")
    print(f"Train MSE: {train_mse}, Test MSE: {test_mse}")
    print(f"Train R2 Score: {train_r2}, Test R2 Score: {test_r2}")
    print("-" * 30)


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Initialize Ridge Regression model
ridge = Ridge(alpha=1.0, random_state=42)

# Fit the model to the training data
ridge.fit(X_train_scaled, y_train)

# Predict on training and test data
y_train_pred_ridge = ridge.predict(X_train_scaled)
y_test_pred_ridge = ridge.predict(X_test_scaled)

# Calculate MSE and R2 for training and test data
train_mse_ridge = mean_squared_error(y_train, y_train_pred_ridge)
test_mse_ridge = mean_squared_error(y_test, y_test_pred_ridge)
train_r2_ridge = r2_score(y_train, y_train_pred_ridge)
test_r2_ridge = r2_score(y_test, y_test_pred_ridge)

# Create scatter plot for training and test data
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred_ridge, label='Training Data', alpha=0.5)
plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], 'r--', label='Perfect Prediction')
plt.title('Ridge Regression - Training Data Predictions')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred_ridge, label='Test Data', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label='Perfect Prediction')
plt.title('Ridge Regression - Test Data Predictions')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Train MSE: {train_mse_ridge}, Test MSE: {test_mse_ridge}")
print(f"Train R2 Score: {train_r2_ridge}, Test R2 Score: {test_r2_ridge}")


In [ ]:
from sklearn.model_selection import cross_val_score

# Initialize Ridge Regression model
ridge = Ridge(alpha=1.0)

# Perform 5-fold cross-validation
cross_val_scores_r2 = cross_val_score(ridge, X_train_scaled, y_train, cv=5, scoring='r2')
cross_val_scores_mse = cross_val_score(ridge, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Calculate mean and standard deviation of scores
mean_r2 = np.mean(cross_val_scores_r2)
std_r2 = np.std(cross_val_scores_r2)
mean_mse = -np.mean(cross_val_scores_mse)  # Negate because scikit-learn returns negative MSE
std_mse = np.std(cross_val_scores_mse)

print(f"Cross-Validation R2: {mean_r2} +/- {std_r2}")
print(f"Cross-Validation MSE: {mean_mse} +/- {std_mse}")


In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('combined_data_zero_sentiment.csv')

# Display the first few rows
data.head()
# Drop rows with missing Ethereum data
data.dropna(subset=['Open_ETH', 'High_ETH', 'Low_ETH', 'Close_ETH', 'Adj Close_ETH', 'Volume_ETH'], inplace=True)

# Fill 'Tweet' column with a placeholder
data['Tweet'].fillna('No Tweet', inplace=True)

# Calculate daily price change for Bitcoin and Ethereum
data['BTC_Daily_Change'] = data['Close'] - data['Open']
data['ETH_Daily_Change'] = data['Close_ETH'] - data['Open_ETH']

# Compute a moving average for the prices of Bitcoin and Ethereum (using a window of 7 days for simplicity)
data['BTC_Moving_Avg_7'] = data['Close'].rolling(window=7).mean()
data['ETH_Moving_Avg_7'] = data['Close_ETH'].rolling(window=7).mean()

# Drop the initial rows where moving average couldn't be computed (first 6 rows)
data.dropna(subset=['BTC_Moving_Avg_7', 'ETH_Moving_Avg_7'], inplace=True)

data.head()
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert 'Tweet' column to numerical data using TF-IDF
vectorizer = TfidfVectorizer(max_features=50)  # Limiting to top 50 features for simplicity
tfidf_matrix = vectorizer.fit_transform(data['Tweet'])

# Convert the matrix to a dataframe to concatenate with our original data
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=data.index)

# Drop the original 'Tweet' column and concatenate the TF-IDF dataframe
data = pd.concat([data.drop(columns=['Tweet']), tfidf_df], axis=1)

# Split the data into features (X) and target (y)
X = data.drop(columns=['Date', 'Close_ETH', 'Adj Close_ETH'])
y = data['Close_ETH']

X.head()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5, :]  # Display the first 5 rows of the scaled training features for inspection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and train the linear regression model
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Predict on both training and test data
y_train_pred = lr.predict(X_train_scaled)
y_test_pred = lr.predict(X_test_scaled)

# Evaluate the model's performance
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_mse, test_mse, train_r2, test_r2

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the training dataset
data = pd.read_csv('combined_data_zero_sentiment.csv')

# Drop rows with missing Ethereum data
data.dropna(subset=['Open_ETH', 'High_ETH', 'Low_ETH', 'Close_ETH', 'Adj Close_ETH', 'Volume_ETH'], inplace=True)

# Drop the 'Tweet' column
data.drop(columns=['Tweet'], inplace=True)

# Split the data into features (X) and target (y)
X = data.drop(columns=['Date', 'Close_ETH', 'Adj Close_ETH'])
y = data['Close_ETH']

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the linear regression model
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Predict on both training and test data
y_train_pred = lr.predict(X_train_scaled)
y_test_pred = lr.predict(X_test_scaled)

# Evaluate the model's performance
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_mse, test_mse, train_r2, test_r2


In [ ]:
# Load the test dataset
test_data = pd.read_csv('Testing.csv')

# Prepare the test data (using only the first row)
test_row = test_data.iloc[0:1]
test_row.drop(columns=['Date', 'Close_ETH', 'Adj Close_ETH', 'Tweet'], inplace=True)
test_row_scaled = scaler.transform(test_row)

# Predict the value
predicted_value = lr.predict(test_row_scaled)

predicted_value

In [ ]:
1625.185+719.18

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

# Load the training dataset
data = pd.read_csv('combined_data_zero_sentiment.csv')
data.dropna(subset=['Open_ETH', 'High_ETH', 'Low_ETH', 'Close_ETH', 'Adj Close_ETH', 'Volume_ETH'], inplace=True)

# Load the testing dataset
test_data = pd.read_csv('Testing.csv')

# Feature Engineering
data['BTC_Daily_Change'] = data['Close'] - data['Open']
data['ETH_Daily_Change'] = data['Close_ETH'] - data['Open_ETH']

test_data['BTC_Daily_Change'] = test_data['Close'] - test_data['Open']
test_data['ETH_Daily_Change'] = data['Close_ETH'] - data['Open_ETH']
# Train-Test Split
X = data.drop(columns=['Date', 'Close_ETH', 'Adj Close_ETH', 'Tweet'])
y = data['Close_ETH']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Training and Evaluation
models = {'Linear Regression': LinearRegression(), 'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42), 'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42), 'XGBoost': XGBRegressor(n_estimators=100, random_state=42)}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    print(f"Model: {name}")
    print(f"Train MSE: {train_mse}, Test MSE: {test_mse}")
    print(f"Train R2 Score: {train_r2}, Test R2 Score: {test_r2}")
    print("-" * 30)
    
    # Plotting
    plt.scatter(y_train, y_train_pred, alpha=0.5, label='Train Data')
    plt.scatter(y_test, y_test_pred, alpha=0.5, label='Test Data')
    plt.title(f'{name} Predictions')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend()
    plt.show()

# Predict the value for the first row in the test dataset
test_data_scaled = scaler.transform(test_data.drop(columns=['Date', 'Close_ETH', 'Adj Close_ETH','Tweet']))
predicted_value = models['Linear Regression'].predict(test_data_scaled[:1])[0]
print(f"Predicted value for the first row in the test dataset: {predicted_value}")

In [ ]:
1598.16-1719.86